In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from wip.hugo import src
from wip.hugo.src import debias
DIR = os.getcwd()

# Data loading & cleaning

In [ ]:
clip_res, bias_res = [], []
clip_res.append(pd.read_csv(os.path.join(DIR, "untrained_test_clip_audit_results.csv"), skipinitialspace=True).rename(columns=lambda x: str(x).strip()))
clip_res.append(pd.read_csv(os.path.join(DIR, "clip_results.csv"), skipinitialspace=True).rename(columns=lambda x: str(x).strip()))
bias_res.append(pd.read_csv(os.path.join(DIR, "untrained_test_bias_results.csv"), skipinitialspace=True).rename(columns=lambda x: str(x).strip()))
bias_res.append(pd.read_csv(os.path.join(DIR, "bias_results.csv"), skipinitialspace=True).rename(columns=lambda x: str(x).strip()))
for dfs in (clip_res, bias_res):
    for df in dfs[1:]:
        dfs[0] = dfs[0].append(df)
clip_res, bias_res = clip_res[0].drop(columns=["Unnamed: 0"]).reset_index(drop=True), bias_res[0].drop(columns=["Unnamed: 0"]).reset_index(drop=True)

import re
def name_formatted(val: str):
    val = val.strip()
    if val.startswith("best_"):
        v = val.split("neptune_run_OXVLB-")[1]
        r, e, s = tuple(re.findall(r"\d+", v))
        return f"trained_{r}"
    else: return f"untrained_{val}"
bias_res["model_name"] = bias_res["model_name"].map(name_formatted)
clip_res["model_name"] = clip_res["model_name"].map(name_formatted)
bias_res["trained"] = bias_res["model_name"].str.startswith("trained_")
clip_res["trained"] = clip_res["model_name"].str.startswith("trained_")
bias_res["long_model_name"] = bias_res["model_name"]
bias_res["model_name"] = bias_res["long_model_name"].map(lambda x: "_".join(x.split("_")[1:]))


In [ ]:
bias_res

# Plotting Code

In [ ]:
aggregated_means = {}
debias_classes = ["gender", "race"]
metrics = ["ndkl", "maxskew"]
datasets = ["FairFaceVal", "UTKFaceVal", "COCOGenderVal"]

import numpy as np
filtered_bias_res = bias_res[bias_res.isin({"evaluation": metrics, "debias_class": debias_classes, "dataset": datasets})[["evaluation", "debias_class", "dataset"]].all(axis=1)].copy()
filtered_bias_res["std_perc"] = filtered_bias_res["std"]/filtered_bias_res["mean"]
col_times = filtered_bias_res.dataset.unique()
fg = sns.FacetGrid(filtered_bias_res, col="evaluation", row="debias_class", margin_titles=True, hue="dataset", height=8, aspect=1.2)
fg.map(sns.barplot, "dataset", "std_perc", order=col_times)
fg.add_legend()
plt.show()

for t in (True, False):
    t_filtered_bias_res = filtered_bias_res[filtered_bias_res["trained"] == t]
    col_times = t_filtered_bias_res.model_name.unique()
    fg = sns.FacetGrid(t_filtered_bias_res, col="evaluation", row="debias_class", margin_titles=True, hue="model_name", height=4, aspect=2)
    fg.map(sns.barplot, "model_name", "std_perc", order=col_times)
    fg.add_legend()
    plt.show()

In [ ]:

# bias prompts
train_prompts = {}
for debias_class in debias_classes:
    _train_prompts = ["a photo of a {} person"]
    _test_prompts = ["a photo of a {} person", "a cropped photo of a {} face"]
    _paired_prompts = [('good'       , 'evil'),
                       ('smart'      , 'dumb'),
                       ('attractive' , 'unattractive'),
                       ('lawful'     , 'criminal'),
                       ('friendly'   , 'unfriendly'),
                       ('clever'     , 'stupid'),
                       ('heroic'     , 'villainous'),
                       ('hardworking', 'lazy'),
                       ('trustworthy', 'dangerous'),
                       ('caring'     , 'nasty')]
    _prompts_n_train = len(_paired_prompts)
    prompt_cfg = debias.prepare_prompt_cfg(debias_class, _paired_prompts, _train_prompts, _test_prompts, _prompts_n_train, test_on_train=False)
    train_prompts[debias_class] = {"train_prompts:": prompt_cfg["BIAS_TRAIN_PROMPTS"], "test_prompts": prompt_cfg["BIAS_TEST_PROMPTS"]}


train_templates = _train_prompts
test_templates = src.datasets.PROMPT_TEMPLATES["prompt_iterations"][src.datasets.PROMPT_TEMPLATES["prompt_iterations"]["group"]=="pairwise_adjectives"]["template"].tolist() + _test_prompts
test_templates = list(set(test_templates))
with open(os.path.join(src.PATHS.IAT.PROMPTS, "pairwise_adjectives.csv"), mode="r") as fill_file:
    fills = []
    for line in fill_file:
        fills.extend([x.strip() for x in line.split(",")])
    for pair in _paired_prompts:
        fills.extend(list(pair))
    fills = list(set(fills))
fills, train_templates, test_templates

